In [26]:
import os
import re
import sys
import time
import shutil
import certifi
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Scrapping
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Exception Error Handling
import socket
import warnings
warnings.filterwarnings("ignore")

# current directory
cur_dir = os.path.dirname(os.path.realpath('__file__'))


if getattr(sys, 'frozen', False) and hasattr(sys, '_MEIPASS'):
    base_path = sys._MEIPASS
    tbl_cache = os.path.join(base_path, 'tbl_cache_')
    conn_path = os.path.join(base_path, 'conn.txt')
    
else:
    root = os.path.abspath(os.path.join(cur_dir, os.pardir, os.pardir))
    src = os.path.abspath(os.path.join(cur_dir, os.pardir))
    sys.path.append(root)
    sys.path.append(src)
    tbl_cache = os.path.join(root, 'tbl_cache')
    conn_path = os.path.join(src, 'gui', 'conn.txt')
    
from scraping import scraper

In [27]:
# info_list = []
# page = 0
# while True:
#     page += 1
#     url = f'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&fltDispCatNo=&prdSort=01&pageIdx={page}&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100070015_Small'
    
#     wd = get_url(url)
#     scroll_down(wd)
#     html = wd.page_source
#     review_soup = BeautifulSoup(html, 'lxml')

#     if review_soup.find('p', 'cate_info_tx').find('span').text.strip() == '0':
#         wd.quit()
#         break

#     # 전체 프로덕트 리스트
#     prd_list = review_soup.find_all('div', 'prd_info')

#     for prd in prd_list:

#         # 올리브영 프로덕트 코드
#         product_code =prd.find('a', 'prd_thumb goodsList')['data-ref-goodsno'] 

#         # 상품 url
#         url = prd.find('a')['href']

#         # 상품명
#         product_name = prd.find('p', 'tx_name').text

#         # 브랜드명
#         brand_name = prd.find('span', 'tx_brand').text

#         # 정상가
#         price = prd.find('span', 'tx_num').text.replace(',', '')

#         # 세일가
#         sale_price = prd.find('span', 'tx_cur').find('span', 'tx_num').text.replace(',', '')
        
#         info_list.append((product_code, product_name, url, brand_name, price, sale_price, 'beauty'))

#     wd.quit()
    
# info_df = pd.DataFrame(info_list, columns = ['product_code', 'product_name', 'product_url', 'brand_name', 'price',
#                                             'sale_price', 'group_details'])

# info_df.to_csv(tbl_cache + '/info_df.csv', index=False)

___
### GlowPick Scraping Test

In [28]:
''' 글로우픽 상품정보 코드 스크레이핑 '''

def parsing(wd, brand, last=None):
    html = wd.page_source
    soup = BeautifulSoup(html,'lxml')
    
    if soup.find("div", class_="search-keywords__contents contents") != None:
        products = soup.find("div", class_="search-keywords__contents contents")
    if products.find("div", class_="products") != None:
        _products = products.find("div", class_="products")
    if _products.find_all("div", class_="products__product product") != None:
        product_list = _products.find_all("div", class_="products__product product")
    
    if len(product_list) == 0:
        return None
    
    else:
        product_id = product_list[-1].get('id')
        if last == 1:
            product_codes = []
            for product in product_list:
                _brand = product.find("span", class_="details__brand__title").text
                if brand == _brand:
                    product_id = product.get('id')
                    product_code = int(product_id[8:])
                    product_codes.append(product_code)
            product_codes = list(set(product_codes))        
            return  product_codes
        
        else:    
            return product_id

In [31]:
def scrap_gl_prd_codes(url, brand):
    # get url 
    wd = scraper.get_url(url)
    if wd == None:
        product_codes = []
        status = -1 # status when parsing url fails
        pass

    else:
        time.sleep(3)
        elm = wd.find_element_by_xpath('//*[@id="default-layout"]/div/div[2]/div/div/div/input')
        elm.send_keys(brand)
        time.sleep(2.5)

        # 팝업창 닫기 
        try:
            popup = wd.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[1]')
            popup.click()
            time.sleep(2.5)
        except:
            pass

        # 검색 (push Enter)
        elm.send_keys(Keys.RETURN)
        time.sleep(5)

        # product_codes = []
        product_id, last_id = None, None
        cnt = 0
        while True:
            product_id = parsing(wd, brand)
            if product_id == last_id:
                cnt += 1
            elif product_id == None:
                product_codes = []
                break
            else:
                pass
            
            last_id = product_id
            tag = wd.find_element_by_id(last_id)
            action = webdriver.ActionChains(wd)
            action.move_to_element(tag).perform()
            time.sleep(7)

            # product_id랑 last_id가 5번 같으면 스크롤 맨 아래에 왔다고 판단
            if cnt == 5:
                product_codes = parsing(wd, brand, last=1)
                wd.quit()
                break
            
    return product_codes

In [30]:
df = pd.read_csv('/Users/mycelebs_95/Downloads/glowpick_product_info_final_version.csv').iloc[:, :7]
_df = df.groupby('brand_name').count().sort_values('id', ascending=False)
brands = _df.index

In [71]:
url = "https://www.glowpick.com/searches/keywords"
brand_dict = {}
for brand in brands[0:10]:
    count = _df.loc[brand].values[0]
    codes = scrap_gl_prd_codes(url, brand)
    brand_dict[brand] = codes
    



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


...

Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (X11; OpenBSD amd64; rv:28.0) Gecko/20100101 Firefox/28.0


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [10]:
url = "https://www.glowpick.com/searches/keywords"
brand_dict = {}
for brand in tqdm(brands[150:155]):
    count = _df.loc[brand].values[0]
    codes = scrap_gl_prd_codes(url, brand)
    brand_dict[brand] = codes
    

  0%|          | 0/5 [00:00<?, ?it/s]

====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
 20%|██        | 1/5 [01:46<07:05, 106.39s/it]

====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


...

Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
 60%|██████    | 3/5 [06:48<04:29, 134.88s/it]

====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
 80%|████████  | 4/5 [08:39<02:05, 125.81s/it]

====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
100%|██████████| 5/5 [10:27<00:00, 125.52s/it]


In [11]:
df = pd.DataFrame(columns=['brand', 'count'])
keys = list(brand_dict.keys())
for key in keys:
    df.loc[len(df), :] = key, len(brand_dict[key])
df

brand count
0   부르조아   116
1     문샷   117
2  케라스타즈   117
3   이브로쉐   118
4   데코르테   116

In [9]:
_df.iloc[150:155]

id  product_name
brand_name                   
부르조아        116           116
문샷          116           116
케라스타즈       115           115
이브로쉐        115           115
데코르테        115           115

___
### Naver Beauty Scraping Test

In [16]:
import os
import re
import sys
import time
import shutil
import certifi
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Scrapping
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Exception Error Handling
import socket
import warnings
warnings.filterwarnings("ignore")

# current directory
cur_dir = os.path.dirname(os.path.realpath('__file__'))


if getattr(sys, 'frozen', False) and hasattr(sys, '_MEIPASS'):
    base_path = sys._MEIPASS
    tbl_cache = os.path.join(base_path, 'tbl_cache_')
    conn_path = os.path.join(base_path, 'conn.txt')
    
else:
    root = os.path.abspath(os.path.join(cur_dir, os.pardir, os.pardir))
    src = os.path.abspath(os.path.join(cur_dir, os.pardir))
    sys.path.append(root)
    sys.path.append(src)
    tbl_cache = os.path.join(root, 'tbl_cache')
    conn_path = os.path.join(src, 'gui', 'conn.txt')

# import module inside package
from hangle import _distance

def get_url(url):
    ''' Set up webdriver, useragent & Get url '''
    
    wd = None
    socket.setdefaulttimeout(30)
    attempts = 0 # url parsing 시도횟수
    # 10번 이상 parsing 실패시 pass
    while attempts < 10:
        try:  
            # user agent
            options = Options() 
            ua = UserAgent(verify_ssl=False)
            userAgent = ua.chrome
            print(userAgent)
            options.add_argument('headless')
            options.add_argument('window-size=1920x1080')
            options.add_argument("disable-gpu")
            options.add_argument('--disable-extensions')
            options.add_argument('--blink-settings=imagesEnabled=false')
            options.add_argument(f'user-agent={userAgent}')

            # web driver 
            wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
            wd.get(url)
            time.sleep(1.5)
            break

        # 예외처리
        except Exception as e:
            try:
                wd.quit()
            except:
                pass
            print(f'\n\n<Error>\n{e}\n\n')
            time.sleep(30)

            # # TLS CA certificate bundle Error (인증서 업데이트 필요)
            # Error_msg = "Could not find a suitable TLS CA certificate bundle"
            # e = str(e)
            # if Error_msg in e:
            #     # cacert.pem update
            #     _certifi = certifi.where()
            #     dst = "/".join(_certifi.split('/')[:-1])
            #     if os.path.isfile(_certifi):
            #         os.remove(_certifi)
            #         print("\n\nremove: ", _certifi, "\n\n")
            #     os.system('curl -k -O https://curl.haxx.se/ca/cacert.pem')
            #     cacert = 'cacert.pem'
            #     shutil.move(cacert, dst)
        attempts += 1
        
    return wd

def scroll_down(wd):
    ''' 
    Scroll down to the bottom of the page 
    ** 데스크탑 웹 페이지에서만 사용가능 **
    '''
    
    prev_height = wd.execute_script("return document.body.scrollHeight")
    while True:
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        wd.implicitly_wait(5)
        time.sleep(1)
        current_height = wd.execute_script("return document.body.scrollHeight")

        if prev_height == current_height:
            break
        prev_height = current_height




def scraper_nv(product_id, search_word):
    ''' 네이버 뷰티윈도 가격비교탭에서 검색어 입력해서 상품 정보 스크레이핑 '''
    
    input_data = search_word.replace('[단종]', '') # '[단종]' 제거 
    input_txt = re.sub(r'[\(\)\{\}\[\]\/]', ' ', input_data) # bracket 제거
    input_txt_ = re.sub(r' +', ' ', input_txt).strip() # 다중 스페이스 제거
    input_keyword = input_txt_.replace(' ','%20') # 쿼리 내 인터벌
    # set up url 
    url = f'https://search.shopping.naver.com/search/all?&frm=NVSHCAT&origQuery={input_keyword}%20%20%20-세트%20-리필%20-set%20-Set%20-SET%20-패키지%20-페키지%20-Package%20-PACKAGE&pagingIndex=1&pagingSize=40&productSet=model&query={input_keyword}&sort=rel&timestamp=&viewType=list&xq=세트%20리필%20set%20Set%20SET%20패키지%20페키지%20Package%20PACKAGE'

    # get url 
    wd = get_url(url)
    if wd == None:
        scraps = []
        status = -1 # status when parsing url fails
        pass
    
    else:
        # scroll down
        scroll_down(wd)
        html = wd.page_source
        soup = BeautifulSoup(html,'lxml') 
        item_divs = soup.find_all('div',class_='basicList_inner__eY_mq')

        scraps = []
        cnt = 0
        for item_div in item_divs:

            # product name
            product_name = item_div.find('div',class_='basicList_title__3P9Q7').text 

            # 문자열 유사도 스코어 확인 (levenshtein distance 활용)
            # input 값이 title과 50%이상 일치하면 수집 
            word_0 = input_txt_.replace(' ', '')
            word_1 = product_name.replace(' ', '')
            cost = _distance.jamo_levenshtein(word_0, word_1)
            max_len = max(len(word_0), len(word_1))
            sim = (max_len - cost) / max_len
            
            if sim >= 0.5 and '세트' not in product_name:
                # product_url
                product_url = item_div.find('a')['href']

                # price
                if item_div.find('span',class_='price_num__2WUXn') != None:
                    price = item_div.find('span',class_='price_num__2WUXn').text 
                else:
                    price = np.nan

                # category
                if item_div.find('div',class_='basicList_depth__2QIie') != None:
                    category = item_div.find('div',class_='basicList_depth__2QIie')
                    category_ = [ctg.text for ctg in category] 

                # product_description
                if item_div.find('div',class_='basicList_detail_box__3ta3h') != None:
                    descriptions = item_div.find('div',class_='basicList_detail_box__3ta3h')
                    if descriptions.find('a', class_='basicList_detail__27Krk') != None:
                        descriptions_ = descriptions.text.split('|')
                        desc_dict = {}
                        for desc in descriptions_:
                            key = desc.split(':')[0].replace(' ', '')
                            value = desc.split(':')[1].replace(' ', '')    
                            desc_dict[key] = value
                        product_description = str(desc_dict)

                    elif descriptions.text != '':
                        desc_dict = {}
                        desc = descriptions.text
                        key = desc.split(':')[0].replace(' ', '')
                        value = desc.split(':')[1].replace(' ', '')    
                        desc_dict[key] = value
                        product_description = str(desc_dict)
                    else:
                        product_description = np.nan

                else:
                    product_description = np.nan

                # registered_date (모든 제품은 등록일을 가지고 있고, 등록일은 동일 클래스 태그 중 맨 항상 맨 앞에 위치)
                if item_div.find('span',class_='basicList_etc__2uAYO') != None: 
                    registered_date = item_div.find('span',class_='basicList_etc__2uAYO').text
                    registered_date_ = registered_date.split('등록일')[-1].rstrip('.')
                else:
                    registered_date_ = np.nan

                # product_reviews_count
                if item_div.find_all('a',class_='basicList_etc__2uAYO') != []:
                    url_boxes = item_div.find_all('a',class_='basicList_etc__2uAYO')
                    url_box = [x for x in url_boxes if '리뷰' in x.text]
                    if len(url_box) != 0:
                        product_reviews_count = url_box[0].find('em',class_='basicList_num__1yXM9').text.replace(',', '')
                    else:
                        product_reviews_count = 0
                else:
                    product_reviews_count = 0

                # product_rating
                if item_div.find('span',class_='basicList_star__3NkBn') != None:
                    product_rating = float(item_div.find('span',class_='basicList_star__3NkBn').text.split('별점')[-1])
                else:
                    product_rating = np.nan

                # product_store
                if item_div.find_all('span',class_='basicList_mall_name__1XaKA') != []:
                    product_store = item_div.find_all('span',class_='basicList_mall_name__1XaKA')[0].text # 최저가 판매처
                else:
                    product_store = np.nan
                
                scraps.append([int(product_id), str(input_txt_), str(product_name), str(product_url), str(price), str(category_), str(product_description), str(registered_date_), int(product_reviews_count), float(product_rating), str(product_store), float(round(sim, 4))])
                cnt += 1
                
            else:
                pass
            
            if cnt == 5:
                break
            
        wd.quit()
        if len(scraps) == 0:
            status = 0
            print(f"\n\t <Not Found>\n\t{input_txt_}\n")
        else:
            status = 1
            
    return scraps, status


In [6]:
df = pd.read_csv('/Users/mycelebs_95/Desktop/projects/tbl_cache/deprepro_0.csv')
df.loc[:, 'search_word'] = df.brand_name + ' ' + df.product_name
df_sample = df.sample(10).reset_index(drop=True)

In [7]:
# for idx in range(len(df_sample)):
#     if str(df_sample.loc[idx, 'search_word']) == "nan":
#             df_sample.loc[idx, 'search_word'] = df_sample.loc[idx, 'brand_name'] + " " + df_sample.loc[idx, 'product_name']

In [8]:
scraps, status = [], []
for idx in tqdm(range(len(df_sample))):
    id_ = df_sample.loc[idx, 'id']
    word = df_sample.loc[idx, 'search_word']
    outputs = scraper_nv(id_, word)
    scraps += outputs[0]
    status.append([id_, word, outputs[1]])

  0%|          | 0/10 [00:00<?, ?it/s]

====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
 10%|█         | 1/10 [00:09<01:24,  9.37s/it]

====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome



	 <Not Found>
	마리꼬 라이트 모이스춰 라이징 크림

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
 20%|██        | 2/10 [00:19<01:17,  9.71s/it]

====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


...


	 <Not Found>
	조말론 미모사 앤 카다멈 바디크림

Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
 90%|█████████ | 9/10 [01:21<00:08,  8.97s/it]

====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36


Driver [/Users/mycelebs_95/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
100%|██████████| 10/10 [01:32<00:00,  9.29s/it]


	 <Not Found>
	렌즈타운 뷰티아이 브라운

